In [ ]:
# SDES

KeyLength = 10
SubKeyLength = 8
DataLength = 8
FLength = 4
 
# Tables for initial and final permutations (b1, b2, b3, ... b8)
IPtable = (2, 6, 3, 1, 4, 8, 5, 7)
FPtable = (4, 1, 3, 5, 7, 2, 8, 6)
 
# Tables for subkey generation (k1, k2, k3, ... k10)
P10table = (3, 5, 2, 7, 4, 10, 1, 9, 8, 6)
P8table = (6, 3, 7, 4, 8, 5, 10, 9)
 
# Tables for the fk function
EPtable = (4, 1, 2, 3, 2, 3, 4, 1)
S0table = (1, 0, 3, 2, 3, 2, 1, 0, 0, 2, 1, 3, 3, 1, 3, 2)
S1table = (0, 1, 2, 3, 2, 0, 1, 3, 3, 0, 1, 0, 2, 1, 0, 3)
P4table = (2, 4, 3, 1)
 
def perm(inputByte, permTable):
    """Permute input byte according to permutation table"""
    outputByte = 0
    for index, elem in enumerate(permTable):
        if index >= elem:
            outputByte |= (inputByte & (128 >> (elem - 1))) >> (index - (elem - 1))
        else:
            outputByte |= (inputByte & (128 >> (elem - 1))) << ((elem - 1) - index)
    return outputByte
 
def ip(inputByte):
    """Perform the initial permutation on data"""
    return perm(inputByte, IPtable)
 
def fp(inputByte):
    """Perform the final permutation on data"""
    return perm(inputByte, FPtable)
 
def swapNibbles(inputByte):
    """Swap the two nibbles of data"""
    return (inputByte << 4 | inputByte >> 4) & 0xff
 
def keyGen(key):
    """Generate the two required subkeys"""
    def leftShift(keyBitList):
        """Perform a circular left shift on the first and second five bits"""
        shiftedKey = [None] * KeyLength
        shiftedKey[0:9] = keyBitList[1:10]
        shiftedKey[4] = keyBitList[0]
        shiftedKey[9] = keyBitList[5]
        return shiftedKey
 
    # Converts input key (integer) into a list of binary digits
    keyList = [(key & 1 << i) >> i for i in reversed(range(KeyLength))]
    permKeyList = [None] * KeyLength
    for index, elem in enumerate(P10table):
        permKeyList[index] = keyList[elem - 1]
    shiftedOnceKey = leftShift(permKeyList)
    shiftedTwiceKey = leftShift(leftShift(shiftedOnceKey))
    subKey1 = subKey2 = 0
    for index, elem in enumerate(P8table):
        subKey1 += (128 >> index) * shiftedOnceKey[elem - 1]
        subKey2 += (128 >> index) * shiftedTwiceKey[elem - 1]
    return (subKey1, subKey2)
 
def fk(subKey, inputData):
    """Apply Feistel function on data with given subkey"""
    def F(sKey, rightNibble):
        aux = sKey ^ perm(swapNibbles(rightNibble), EPtable)
        index1 = ((aux & 0x80) >> 4) + ((aux & 0x40) >> 5) + \
                 ((aux & 0x20) >> 5) + ((aux & 0x10) >> 2)
        index2 = ((aux & 0x08) >> 0) + ((aux & 0x04) >> 1) + \
                 ((aux & 0x02) >> 1) + ((aux & 0x01) << 2)
        sboxOutputs = swapNibbles((S0table[index1] << 2) + S1table[index2])
        return perm(sboxOutputs, P4table)
 
    leftNibble, rightNibble = inputData & 0xf0, inputData & 0x0f
    return (leftNibble ^ F(subKey, rightNibble)) | rightNibble
 
def encrypt(key, plaintext):
    """Encrypt plaintext with given key"""
    data = fk(keyGen(key)[0], ip(plaintext))
    return fp(fk(keyGen(key)[1], swapNibbles(data)))
 
def decrypt(key, ciphertext):
    """Decrypt ciphertext with given key"""
    data = fk(keyGen(key)[1], ip(ciphertext))
    return fp(fk(keyGen(key)[0], swapNibbles(data)))  
"""-------end of SDES implementation-------"""

"""Method for double encryption taking key1,key2 and 
 plaintext and returning corresponding ciphertext"""
 
def double_encrypt(k1,k2,plaintext):
    temp_cipher = encrypt(k1,plaintext)
    ciphertext = encrypt(k2,temp_cipher)
    return ciphertext

"""Method for double decryption taking key1,key2 and 
ciphertext and returning corresponding plaintext"""

def double_decrypt(k1,k2,cipher):
    middle_cipher = decrypt(k2,cipher)
    plain_text = decrypt(k1,middle_cipher)
    return plain_text

""" asking user to input plaintext in order to get 
  (plaintext,cipherext) pairs to attack"""

plaintext=list()
cipher_text=list()
for i in range(0,6):
    text = int(input("Enter "+ str(i) +" text to encrypt: "))
    plaintext.append(text)

key1 = int(input("Enter key1: "))
key2 = int(input("Enter key2:" ))
for i in range(0,6):
    cipher = double_encrypt(key1,key2,plaintext[i])
    cipher_text.append(cipher)
    print("The cipher for the given input"+ str(i) +" is: ",cipher_text[i])
for i in range(0,1023):
    for j in range(0,1023):
        cipher0=double_encrypt(i,j,plaintext[0])
        cipher1=double_encrypt(i,j,plaintext[1])
        cipher2=double_encrypt(i,j,plaintext[2])
        cipher3=double_encrypt(i,j,plaintext[3])
        cipher4=double_encrypt(i,j,plaintext[4])
        cipher5=double_encrypt(i,j,plaintext[5])
        if cipher0==cipher_text[0] and cipher1==cipher_text[1] and cipher2==cipher_text[2] and cipher3==cipher_text[3] and cipher4==cipher_text[4] and cipher5==cipher_text[5]:
            print("Keys k1 and k2 are",i,j)

In [ ]:
print(cipher_text[0])